In [1]:
import time

In [2]:
from tensorflow.keras.models import load_model

2024-04-09 04:29:51.244841: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-09 04:29:51.844896: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from deap import base, creator, tools, algorithms

In [4]:
import numpy as np

In [5]:
from tqdm import tqdm

In [6]:
import tensorflow as tf

In [7]:
# import multiprocessing

In [8]:
loaded_model = load_model('surogate.h5')

2024-04-09 04:29:53.439543: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9604 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
2024-04-09 04:29:53.440102: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9621 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [9]:
# 特定频率
def loss_freq(s_para, disp=False):
    # 能量
    E = np.square(s_para)
    E11 = E[:,0] + E[:,1]
    E21 = E[:,2] + E[:,3]
    E31 = E[:,4] + E[:,5]
    E41 = E[:,6] + E[:,7]
    P21 = np.arctan2(s_para[:,3], s_para[:,2])
    P31 = np.arctan2(s_para[:,5], s_para[:,4])

    # 压制
    loss1 = E11 - E21 - E31 + E41
    # 比例
    loss2 = np.abs(E21 / (E31 + E21) - 0.7)
    # phase
    loss3 = np.abs(P21 - P31 - np.pi / 4)

    loss4 = np.abs(np.sum(E, axis = -1) - 1)

    if disp:
        print(f"P21:{P21}\nP31:{P31}")
        print(f"E11:{E11}\nE21:{E21}\nE31:{E31}\nE41:{E41}")
        print(f"loss1:{loss1}\nloss2:{loss2}\nloss3:{loss3}")
        return E11, E21, E31, E41, P21, P31, loss1, loss2, loss3
    
    return loss1 + loss2 + loss3 + loss4

In [10]:
def dgn_obj(s1, s2, s3):
    # max_loss = np.max([loss_freq(s1), loss_freq(s2), loss_freq(s3)], axis=0)
    # max_loss_tuple = [(value,) for value in max_loss]
    # return max_loss_tuple
    sum_loss = loss_freq(s1) + loss_freq(s2) + loss_freq(s3)
    sum_loss_tuple = [(value,) for value in sum_loss]
    return sum_loss_tuple

In [11]:
def evaluate(x):
    geoms = np.array(x)
    n_samples = geoms.shape[0]
    inputs1 = np.concatenate((freq1[:n_samples], geoms), axis=-1)
    inputs2 = np.concatenate((freq2[:n_samples], geoms), axis=-1)
    inputs3 = np.concatenate((freq3[:n_samples], geoms), axis=-1)
    y_pred1 = loaded_model.predict(inputs1, batch_size=n_samples, verbose=0)
    y_pred2 = loaded_model.predict(inputs2, batch_size=n_samples, verbose=0)
    y_pred3 = loaded_model.predict(inputs3, batch_size=n_samples, verbose=0)
    return dgn_obj(y_pred1, y_pred2, y_pred3)

In [12]:
def checkBounds():
    def decorator(func):
        def wrapper(*args, **kargs):
            offspring = func(*args, **kargs)
            for child in offspring:
                mask = np.where((child < lb) | (child > ub))
                child[mask] = np.random.uniform(lb[mask], ub[mask])
            return offspring
        return wrapper
    return decorator

In [13]:
# 参数数量，迭代次数
EPOCHS = 100

In [14]:
# 物理边界
lb = np.array([1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 4, 4])
ub = np.array([5, 5, 5, 5, 5, 5, 5, 5, 100, 100, 100, 100])

In [15]:
# 初始数据
freq1 = np.ones((40000, 1), dtype=np.float32) * 2.4
freq2 = np.ones((40000, 1), dtype=np.float32) * 2.5
freq3 = np.ones((40000, 1), dtype=np.float32) * 2.6

In [16]:
def myOpti(num_nodes):
    # 定义个体表示方式
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()

    toolbox.register("attr_float", np.random.uniform, low=lb, high=ub)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr_float)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    toolbox.register("evaluate", evaluate)

    # 注册选择、交叉和突变操作
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    toolbox.decorate("mate", checkBounds())
    toolbox.decorate("mutate", checkBounds())

    population = toolbox.population(n=num_nodes)
    halloffame = tools.HallOfFame(1, similar=np.array_equal)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    cxpb = 0.5
    mutpb = 0.2
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.evaluate(invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    halloffame.update(population)
    record = stats.compile(population)
    logbook.record(gen=0, nevals=len(invalid_ind), **record)

    t0 = time.time()
    for gen in tqdm(range(1, EPOCHS + 1)):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))
    
        # Vary the pool of individuals
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.evaluate(invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
    
        # Update the hall of fame with the generated individuals
        halloffame.update(offspring)
    
        # Replace the current population by the offspring
        population[:] = offspring
    
        # Append the current generation statistics to the logbook
        record = stats.compile(population)
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
    return (time.time() - t0) / EPOCHS

In [17]:
logs = []

In [18]:
for i in range(200, 20001, 200):
    logs.append(myOpti(i))

100%|█████████████████████████████████████████| 100/100 [00:11<00:00,  8.93it/s]
/home/delta/miniconda3/envs/jt_ma/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/delta/miniconda3/envs/jt_ma/lib/python3.9/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
100%|█████████████████████████████████████████| 100/100 [01:08<00:00,  1.45it/s]


In [19]:
for log in logs:
    print(log)

0.11203099966049195
0.11755289316177368
0.12413538694381714
0.13165103673934936
0.1362473511695862
0.1425092935562134
0.1477104377746582
0.15186994314193725
0.15852530002593995
0.16294898509979247
0.17084757328033448
0.1743980050086975
0.18101976871490477
0.18404898166656494
0.18990570068359375
0.19561043977737427
0.20106998205184937
0.20828263998031615
0.21352701425552367
0.2185478091239929
0.22742772817611695
0.2320934557914734
0.24035312175750734
0.23988279104232788
0.24801589012145997
0.2533854293823242
0.2632793593406677
0.2665660810470581
0.2741831231117249
0.2785260057449341
0.28472535848617553
0.29059685945510866
0.29597557306289674
0.29906726837158204
0.3031743383407593
0.3129207754135132
0.3200231599807739
0.32096037864685056
0.32584104061126706
0.333216598033905
0.3412554526329041
0.34477975845336917
0.35461256742477415
0.35528130531311036
0.3650142955780029
0.36916797161102294
0.37727360010147093
0.3802617573738098
0.38534990310668943
0.38875378608703615
0.3963348889350891
